In [18]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

In [94]:
class NeuralNet:
    def __init__(self, feats, labels, hidShape, miu=0.1, nItr = 10):
        self.feats= feats
        self.labels=labels
        self.ni = np.shape(feats)[1]
        self.no = np.shape(labels)[1]
        self.hShape = hidShape
        self.miu = miu
        self.nItr = nItr
        
        self.iNodes = np.zeros(self.ni)
        self.oNodes = np.zeros(self.no)
        
        self.ihWeights = np.random.randn(self.ni, self.hShape[0])
        
        self.oBiases = np.zeros(self.no)

        self.hWeights = {}
        for i in range(len(self.hShape)):
            
            if i==len(self.hShape)-1:
                self.hWeights[i]=np.random.randn(self.hShape[i], self.no)
            else:
                self.hWeights[i]=np.random.randn(self.hShape[i], self.hShape[i+1])
        
        self.hNodes = {}
        self.hBiases = {}
        for i in range(len(self.hShape)):
            self.hNodes[i]=np.zeros(self.hShape[i])
            self.hBiases[i]=np.zeros(self.hShape[i])
        
        
    def sigmoid(self, s):
        return 1/(1+np.exp(-s))
    
    def forward(self, x):
        self.iNodes=x
        
        vHid=np.dot(self.iNodes, self.ihWeights)
        
        for i in range(len(self.hShape)):
            self.hNodes[i] = self.sigmoid(vHid)+self.hBiases[i]
            vHid=np.dot(self.hNodes[i],self.hWeights[i])

        self.oNodes=self.sigmoid(vHid)+self.oBiases
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def backward(self):
        sumihDel = 0
        sumhDel = {}
        for i in range(len(self.hShape)):
            if (i==len(self.hShape)-1):
                sumhDel[i]= np.zeros(shape=[self.hShape[i],self.no])
            else:
                sumhDel[i]= np.zeros(shape=[self.hShape[i],self.hShape[i+1]])
        for (x,y) in zip(self.feats,self.labels):
            self.forward(x)
            self.oError = self.oNodes-y
            self.oDelta = self.oError*self.sigmoidPrime(self.oNodes)
             
            self.hError = {}
            self.hDelta = {}
            
            for i in range(len(self.hShape)-1,0,-1):
                if i==len(self.hShape)-1:
                    self.hError[i] = np.dot(self.oDelta, self.hWeights[i].T)
                    self.hDelta[i] = self.hError[i]*self.sigmoidPrime(self.hNodes[i])
                    sumhDel[i] += np.dot(np.array([self.hNodes[i]]).T, np.array([self.oDelta]))
                else:
                    self.hError[i] = np.dot(self.hDelta[i+1], self.hWeights[i].T)
                    self.hDelta[i] = self.hError[i]*self.sigmoidPrime(self.hNodes[i])
                    sumhDel[i] += np.dot(np.array([self.hNodes[i]]).T, np.array([self.hDelta[i+1]]))
            
            if len(self.hShape)>1:
                self.hError[0] = np.dot(self.hDelta[1], self.hWeights[0].T)
            else:
                self.hError[0] = np.dot(self.oDelta, self.hWeights[0].T)
            
            self.hDelta[0] = self.hError[0]*self.sigmoidPrime(self.hNodes[0])
            
            sumihDel += np.dot(np.array([self.iNodes]).T, np.array([self.hDelta[0]]))

        for i in range(len(self.hShape)-1,0,-1):
            self.hWeights[i] += -self.miu*sumhDel[i]
        self.ihWeights += -self.miu*sumihDel
        
    def train(self, iterate=False):
        if iterate:
            for i in range(self.nItr):
                self.backward()
        else:
            loop_count = 0
            conv_count = 0
            div_len = 2
            
            prev_acc=self.accuracy(self.feats,self.labels)
            while True:
                
                self.backward()
                loop_count+=1
                accuracy = self.accuracy(self.feats,self.labels)

                if abs(accuracy-prev_acc)<=div_len:
                    conv_count+=1
                else:
                    conv_count=0
                if conv_count==20:
                    print("Converges after: "+str(loop_count)+" iterations")
                    break
                if loop_count==200:
                    print("accuracy so far: "+str(self.accuracy(self.feats,self.labels)))
                    print(str(loop_count)+" iterations")
                    break
            
                
    def accuracy(self, feats, labels):
        count =0
        for (x,y) in zip(feats,labels):
            self.forward(x)
            if np.argmax(y)==np.argmax(self.oNodes):
                count+=1
        return count/len(self.labels)*100
               

In [96]:

dataset = pd.read_table('./datasets/Train.txt', sep='\s+', skiprows=[0], header=None)
# dataset = pd.read_table('./Perceptron/trainLinearlyNonSeparable.txt', sep='\s+', header=None)
array = dataset.values
train_x = array[:, :-1]
train_y = array[:, -1]

feats = MinMaxScaler().fit_transform(train_x)
labels = OneHotEncoder().fit_transform(train_y.reshape(-1, 1)).toarray()
nFeat=np.shape(feats)[1]
nClass=np.shape(labels)[1]

model = NeuralNet(feats,labels,[2,5,10,4])
# model.forward(x)
# print(model.iNodes)
# print(model.hNodes)
# print(model.oNodes)
model.train()
# print(model.oError)
dataset = pd.read_table('./datasets/Test.txt', sep='\s+', skiprows=[0], header=None)
# dataset = pd.read_table('./Perceptron/testLinearlyNonSeparable.txt', sep='\s+', header=None)
array = dataset.values
train_x = array[:, :-1]
train_y = array[:, -1]

feats = MinMaxScaler().fit_transform(train_x)
labels = OneHotEncoder().fit_transform(train_y.reshape(-1, 1)).toarray()
print(model.accuracy(feats,labels))

accuracy so far: 100.0
200 iterations
98.32775919732441


In [105]:
ar = {}

for i in range(0,-1,-1):
    ar[i]=np.arange(5)

print(ar)

{0: array([0, 1, 2, 3, 4])}


In [66]:
abs(-5)

5